## 1. Instalación librerias necesarias

In [ ]:
!pip install numpy
!pip install pandas



Looking in indexes: https://artifactory.apps.bancolombia.com/api/pypi/pypi-bancolombia/simple



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://artifactory.apps.bancolombia.com/api/pypi/pypi-bancolombia/simple



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://artifactory.apps.bancolombia.com/api/pypi/pypi-bancolombia/simple


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyspark 3.4.1 requires py4j==0.10.9.7, but you have py4j 0.10.9.9 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Importar Librerias

In [37]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

dfOpenSearch = pd.read_excel('Opensearch.xlsx')


columnas_interes = [
    'generaltimestamp', 'integrationplatform', 'environment', 'executionenvironment',
    'technicalservicename', 'statuscode', 'detail', 'causes', 'operation', 'consumerid',
    'messageid', 'version', 'ingestion_year', 'ingestion_month', 'ingestion_day'
]

## 3. Limpieza del dataset
Se organizan los campos


In [38]:
dfOpenSearchL = dfOpenSearch[columnas_interes].copy()

dfOpenSearchL.rename(columns={
    'generaltimestamp': 'Hora_de_transaccion',
    'integrationplatform': 'Herramienta_desarrollo',
    'executionenvironment': 'Plataforma_de_despliegue',
    'technicalservicename': 'Nombre_de_servicio',
    'statuscode': 'Estado_de_respuesta',
    'detail': 'Detalle_de_error',
    'causes': 'Causas',
    'operation': 'Nombre_de_operacion',
    'consumerid': 'Id_consumidor',
    'messageid': 'Id_mensaje',
    'version': 'Version',
    'ingestion_year': 'Año_de_ingestion',
    'ingestion_month': 'Mes_de_ingestion',
    'ingestion_day': 'Dia_de_ingestion',
    'environment': 'Ambiente'
}, inplace=True)


dfOpenSearchL.dropna(how='all', inplace=True)
dfOpenSearchL = dfOpenSearchL[dfOpenSearchL['Ambiente'].str.lower().isin(['production', 'pdn', 'produccion'])]
dfOpenSearchL.drop(columns=['Ambiente'], inplace=True)
dfOpenSearchL.insert(0, 'ID', range(1, len(dfOpenSearchL) + 1))
orden_columnas = [
    'ID', 'Hora_de_transaccion', 'Herramienta_desarrollo', 'Plataforma_de_despliegue',
    'Nombre_de_servicio', 'Estado_de_respuesta', 'Detalle_de_error', 'Causas',
    'Nombre_de_operacion', 'Id_consumidor', 'Id_mensaje', 'Version',
    'Año_de_ingestion', 'Mes_de_ingestion', 'Dia_de_ingestion'
]
dfOpenSearchL = dfOpenSearchL[orden_columnas]

# Exportar a CSV
dfOpenSearchL.to_csv('opensearch_filtrado.csv', index=False, encoding='utf-8')

## 4. Actualización campos del otro dataset

In [41]:
dfInvServiciosYApisPDN= pd.read_excel('Inventario de Servicios y Apis Producción.xlsx', sheet_name='Inventario de Servicios')
#print(list(dfInvServiciosYApisPDN.columns))


columnas_interes = [
    'TIPO', 'Nombre Producto', 'Version Producto', 'NOMBRE DEL SERVICIO',
    'Nombre Tecnico Api', 'COMPONENTE MENOR', 'EVC', 'Nombre Equipo',
    'SERVICIO PROVEEDOR', 'CONSUMIDORES'
]


## 5. Limpieza del otro dataset

In [42]:
dfInv = dfInvServiciosYApisPDN[columnas_interes].copy()

dfInv.rename(columns={
    'TIPO': 'Tipo',
    'Nombre Producto': 'Nombre_producto_API',
    'Version Producto': 'Version_producto',
    'NOMBRE DEL SERVICIO': 'Nombre_de_servicio/API',
    'Nombre Tecnico Api': 'Nombre_Tecnico_API',
    'COMPONENTE MENOR': 'Componente_menor',
    'EVC': 'EVC',
    'Nombre Equipo': 'Nombre_equipo',
    'SERVICIO PROVEEDOR': 'Servicio_proveedor',
    'CONSUMIDORES': 'Consumidores'
}, inplace=True)

dfInv.dropna(how='all', inplace=True)
dfInv['QMANAGER'] = ''
dfInv['Namespace'] = ''
dfInv.insert(0, 'ID', range(1, len(dfInv) + 1))
orden_columnas = [
    'ID', 'Tipo', 'Nombre_producto_API', 'Version_producto', 'Nombre_de_servicio/API',
    'Nombre_Tecnico_API', 'Componente_menor', 'EVC', 'Nombre_equipo',
    'Servicio_proveedor', 'Consumidores', 'QMANAGER', 'Namespace'
]
dfInv = dfInv[orden_columnas]

# Exportar a CSV
dfInv.to_csv('inventario_servicios_apis.csv', index=False, encoding='utf-8')

## 6. crear base de datos e insertar datos en MYSQL

In [14]:
!pip install mysql-connector-python

Looking in indexes: https://artifactory.apps.bancolombia.com/api/pypi/pypi-bancolombia/simple
     ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
     - -------------------------------------- 0.8/16.4 MB 4.8 MB/s eta 0:00:04
     ---- ----------------------------------- 1.8/16.4 MB 5.0 MB/s eta 0:00:03
     ------- -------------------------------- 2.9/16.4 MB 4.8 MB/s eta 0:00:03
     --------- ------------------------------ 3.9/16.4 MB 4.8 MB/s eta 0:00:03
     ------------ --------------------------- 5.0/16.4 MB 4.8 MB/s eta 0:00:03
     -------------- ------------------------- 6.0/16.4 MB 4.8 MB/s eta 0:00:03
     ---------------- ----------------------- 6.8/16.4 MB 4.8 MB/s eta 0:00:03
     ------------------- -------------------- 8.1/16.4 MB 4.8 MB/s eta 0:00:02
     ---------------------- ----------------- 9.2/16.4 MB 4.8 MB/s eta 0:00:02
     ------------------------ --------------- 10.2/16.4 MB 4.8 MB/s eta 0:00:02
     -------------------------- -----------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 7. Importar librerías y definir funciones

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import re
from datetime import datetime, time

# Leer scripts SQL
def leer_sql(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

# Función para normalizar nombres de servicio
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return ''
    nombre = str(nombre)
    nombre = re.sub(r'^[A-Z]{2}\d{6}_', '', nombre)
    nombre = nombre.lower().replace(" ", "").replace("_", "")
    return nombre

# Convertir Hora_de_transaccion a tipo DATETIME válido
def convertir_hora(hora_str):
    try:
        return datetime.strptime(hora_str, "%M:%S.%f").time()
    except:
        return None


## 8.  Leer y procesar archivos CSVs

In [50]:
df_inv = pd.read_csv('inventario_servicios_apis.csv', encoding='utf-8')
df_os = pd.read_csv('opensearch_filtrado.csv', encoding='utf-8')

# Procesar inventario
df_inv = df_inv[[ 
    'Tipo', 'Nombre_producto_API', 'Version_producto', 'Nombre_de_servicio/API',
    'Nombre_Tecnico_API', 'Componente_menor', 'Nombre_equipo',
    'Servicio_proveedor', 'Consumidores'
]].copy()

df_inv.rename(columns={'Nombre_de_servicio/API': 'Nombre_de_servicio_API'}, inplace=True)
df_inv.dropna(how='all', inplace=True)
df_inv['Id'] = range(1, len(df_inv) + 1)
df_inv['nombre_clave'] = df_inv['Nombre_de_servicio_API'].apply(normalizar_nombre)

# Procesar OpenSearch
df_os.columns = [col.strip().replace('Ã±', 'ñ').replace('AÃ±o', 'Año') for col in df_os.columns]
df_os['Id'] = range(1, len(df_os) + 1)
df_os['Hora_de_transaccion'] = df_os['Hora_de_transaccion'].apply(convertir_hora)
df_os['nombre_clave'] = df_os['Nombre_de_servicio'].apply(normalizar_nombre)


## 9.  Relacionar y limpiar DataFrames

In [51]:
# Hacer merge usando la clave normalizada
df_os = df_os.merge(
    df_inv[['Id', 'nombre_clave']],
    on='nombre_clave',
    how='left',
    suffixes=('', '_inv')
)

df_os.rename(columns={'Id_inv': 'Inventario_Id'}, inplace=True)

# Limpiar columnas temporales
df_inv.drop(columns=['nombre_clave'], inplace=True)
df_os.drop(columns=['nombre_clave'], inplace=True)

# Reordenar columnas
df_inv = df_inv[[
    'Id', 'Tipo', 'Nombre_producto_API', 'Version_producto',
    'Nombre_de_servicio_API', 'Nombre_Tecnico_API', 'Componente_menor',
    'Nombre_equipo', 'Servicio_proveedor', 'Consumidores'
]]

df_os = df_os[[
    'Id', 'Hora_de_transaccion', 'Herramienta_desarrollo', 'Plataforma_de_despliegue',
    'Nombre_de_servicio', 'Estado_de_respuesta', 'Detalle_de_error', 'Causas',
    'Nombre_de_operacion', 'Id_consumidor', 'Id_mensaje', 'Version',
    'Año_de_ingestion', 'Mes_de_ingestion', 'Dia_de_ingestion', 'Inventario_Id'
]]


## 10.   Insertar  datos en MySQL

In [56]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import re
from datetime import datetime, time

try:
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='12345'
    )

    if conn.is_connected():
        with conn.cursor() as cursor:
            # Crear base de datos
            script_db = leer_sql('base_datos.sql')
            for stmt in script_db.split(';'):
                if stmt.strip():
                    cursor.execute(stmt)
            print("✅ Base de datos creada")

            # Seleccionar base de datos
            conn.database = 'inventario_datos'

            # Crear tablas
            script_tablas = leer_sql('tablas.sql')
            for stmt in script_tablas.split(';'):
                if stmt.strip():
                    cursor.execute(stmt)
            print("✅ Tablas creadas")

            # Insertar datos en inventario_servicios_apis
            insert_inv = """
                INSERT INTO inventario_servicios_apis (
                    Id, Tipo, Nombre_producto_API, Version_producto,
                    Nombre_de_servicio_API, Nombre_Tecnico_API, Componente_menor,
                    Nombre_equipo, Servicio_proveedor, Consumidores
                ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """
            for _, row in df_inv.iterrows():
                valores = [None if pd.isna(v) else str(v) for v in row]
                cursor.execute(insert_inv, tuple(valores))

            # Insertar datos en opensearch_filtrado
            insert_os = """
                INSERT INTO opensearch_filtrado (
                    Id, Hora_de_transaccion, Herramienta_desarrollo, Plataforma_de_despliegue,
                    Nombre_de_servicio, Estado_de_respuesta, Detalle_de_error, Causas,
                    Nombre_de_operacion, Id_consumidor, Id_mensaje, Version,
                    Año_de_ingestion, Mes_de_ingestion, Dia_de_ingestion, Inventario_Id
                ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """
            for _, row in df_os.iterrows():
                valores = []
                for v in row:
                    if pd.isna(v):
                        valores.append(None)
                    elif isinstance(v, time):  # mantener tipo time como está
                        valores.append(v)
                    else:
                        valores.append(str(v))
                cursor.execute(insert_os, tuple(valores))

            conn.commit()
            print("✅ Datos insertados correctamente")

except Error as e:
    print("❌ Error en MySQL:", e)

except Exception as ex:
    print("❌ Error general:", ex)

finally:
    if conn.is_connected():
        conn.close()


✅ Base de datos creada
✅ Tablas creadas
✅ Datos insertados correctamente
